已连接到 .env (Python 3.10.16)


In [1]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings


from src.llms.siliconflow import SiliconflowLLM
api_key = "key"

llm = SiliconflowLLM(
    api_key=api_key,
    api_base="https://api.siliconflow.cn/v1",
    model_name="Qwen/Qwen2.5-VL-72B-Instruct",
)

# 使用本地部署的ollama模型
llm = Ollama(model="llama3.1", request_timeout=60.0)
Settings.llm = llm


embed_model = OllamaEmbedding(model_name="bge-m3")  # 向量纬度 1024
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 132

In [ ]:
from pymilvus import connections, utility, Collection
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.milvus import MilvusVectorStore
from llama_index.core import StorageContext

# # 1. 确保连接
# connections.connect(alias="default", host="localhost", port="19530")

# # 连接新创建的数据库
# connections.connect("default", host="127.0.0.1", port="19530", db_name="LlamaIndexBase") 

# 3. 创建集合
collection_name = "llama_collection"
dimension = 1024

# 4. 创建向量存储
vector_store = MilvusVectorStore(
    uri="http://localhost:19530",  # 当创建 MilvusVectorStore 时，如果没有指定 uri 参数，它会默认使用本地模式
    collection_name=collection_name,
    dim=dimension,
    overwrite=True,
)

# 5. 加载并插入数据
documents = SimpleDirectoryReader("../data/pdf/").load_data()
print(f"加载了 {len(documents)} 个文档")


# 5. 为文档生成嵌入向量
for doc in documents:
    embedding = Settings.embed_model.get_text_embedding(doc.text)
    doc.embedding = embedding
    print(f"生成嵌入向量，维度: {len(embedding)}")


async def insert_data(documents):
    # 插入数据
    inserted_ids = await vector_store.async_add(documents)
    print(f"成功插入 {len(inserted_ids)} 条数据")

    # 7. 验证插入
    collection = vector_store._collection
    print("\n插入后统计:")
    print(f"集合 llama_collection 中有 {collection.num_entities} 个实体")

    return inserted_ids


await insert_data(documents)


# 6. 强制刷新数据
vector_store.client.flush("llama_collection")

storage_context = StorageContext.from_defaults(vector_store=vector_store)


# load your index from stored vectors
index = VectorStoreIndex.from_vector_store(
    vector_store, storage_context=storage_context
)

# create a query engine
query_engine = index.as_query_engine()
response = query_engine.query("软银数据中心的机架设备统计是什么?")
print(response)


# # 8. 关闭连接
# connections.disconnect("default")
# print("关闭连接")

In [6]:
from notebooks.setup import init_setup_settings
from src.llms.siliconflow import SiliconflowLLM

config = init_setup_settings()

llm = SiliconflowLLM(
    api_key=config.siliconflow.api_key,
    api_base="https://api.siliconflow.cn/v1",
    model_name="Qwen/Qwen2.5-VL-72B-Instruct",
)

response = llm.complete("你好，介绍一下自己？")
print(response.text)  # 使用 flush=True 确保立即输出

你好！我是来自阿里云的大规模语言模型，我叫通义千问。我是阿里云自主研发的超大规模语言模型，能够回答问题、创作文字，还能表达观点、撰写代码。如果您有任何问题或需要帮助，请随时告诉我，我会尽力提供支持。
